In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import time
import requests

In [10]:
mars_nasa_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
mars_jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
mars_twitter_url= "https://twitter.com/marswxreport?lang=en"
mars_facts_url = "https://space-facts.com/mars/"
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [3]:
#Scrape NASA Mars News
def scrape_mars_news():
    chrome_path = "/usr/local/bin/chromedriver"

    browser = webdriver.Chrome(chrome_path)
    browser.get(mars_nasa_url)

    time.sleep(1)

    soup = bs(browser.page_source, 'html.parser')
    tag = soup.find('div', class_='content_title').text
    paragraph = soup.find('div', class_='article_teaser_body').text
    browser.close()

    return tag, paragraph

scrape_mars_news()

("NASA's Mars 2020 Rover Closer to Getting Its Name",
 "155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.")

In [13]:
#Scrape JPL Featured Mars Image
def scrape_jpl_mars_image():
    listings = {}
    chrome_path = "/usr/local/bin/chromedriver"
    browser = webdriver.Chrome(chrome_path)

    browser.get(mars_jpl_url)

    time.sleep(1)

    soup = bs(browser.page_source, 'html.parser')
    img_title = soup.find('div', class_='item_tease_overlay').text
    listings[img_title] = soup.find('img', class_='thumb')['src']
    featured_img_jpg = "https://www.jpl.nasa.gov" + listings[img_title]

    browser.close()
    
    return listings

scrape_jpl_mars_image()

{'Arabia Terra Channel': '/spaceimages/images/wallpaper/PIA23656-640x350.jpg'}

In [17]:
#Scrape Twitter Mars Weather
def scrape_mars_weather():

    response = requests.get(mars_twitter_url)
    soup = bs(response.content, 'html.parser')

    mars_weather = soup.find('p', class_='TweetTextSize').text

    images = soup.find_all('img', class_="css-9pa8cd")
    for i in images:
        print(i)
scrape_mars_weather()


<img alt="" class="avatar size32"/>
<img alt="" class="avatar size32"/>
<img alt="" class="avatar size32"/>
<img alt="" src="https://pbs.twimg.com/profile_banners/786939553/1550640093/1500x500"/>
<img alt="Mars Weather" class="ProfileAvatar-image" src="https://pbs.twimg.com/profile_images/2552209293/220px-Mars_atmosphere_400x400.jpg"/>
<img alt="Mars Weather" class="ProfileCardMini-avatarImage" src="https://pbs.twimg.com/profile_images/2552209293/220px-Mars_atmosphere_normal.jpg"/>
<img alt="" class="avatar js-action-profile-avatar" src="https://pbs.twimg.com/profile_images/2552209293/220px-Mars_atmosphere_bigger.jpg"/>
<img alt="" data-aria-label-part="" src="https://pbs.twimg.com/media/EPfnKPuXkAEjk8X.jpg" style="width: 100%; top: -0px;"/>
<img alt="" class="avatar js-action-profile-avatar" src="https://pbs.twimg.com/profile_images/2552209293/220px-Mars_atmosphere_bigger.jpg"/>
<img alt="" data-aria-label-part="" src="https://pbs.twimg.com/media/EPdBXakWAAANU44.jpg" style="width: 100

In [7]:
#Scrape Mars Facts table
def scrape_mars_facts():
    chrome_path = "/usr/local/bin/chromedriver"
    browser = webdriver.Chrome(chrome_path)

    browser.get(mars_facts_url)

    time.sleep(1)
    soup = bs(browser.page_source, 'html.parser')
    mars_facts_table = soup.find('tbody')

    browser.close()

    return mars_facts_table

scrape_mars_facts()

<tbody><tr class="row-1 odd"><td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/></td></tr><tr class="row-2 even"><td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/></td></tr><tr class="row-3 odd"><td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.39 × 10^23 kg<br/> (0.11 Earths)</td></tr><tr class="row-4 even"><td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/moons/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/moons/deimos/">Deimos</a>)</td></tr><tr class="row-5 odd"><td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km<br/> (1.38 AU)</td></tr><tr class="row-6 even"><td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/></td></tr><tr class="row-7 odd"><td class="column-1"><strong>Surface Temperature: </strong></td><td class="co

In [6]:
#Scrape Mars Hemisphere Images
def scrape_mars_hemisphere():
    mars_hemispheres_entry = {}

    chrome_path = "/usr/local/bin/chromedriver"
    browser = webdriver.Chrome(chrome_path)

    browser.get(mars_hemispheres_url)

    soup = bs(browser.page_source, 'html.parser')
    link_names = soup.find_all('h3')
    for index, data in enumerate(link_names):
        link = browser.find_element_by_link_text(str(data.text))
        link.click()
        time.sleep(1)

        title = browser.find_element_by_class_name("title")
        image_title = title.text

        image = browser.find_element_by_xpath("//a[@target='_blank']")

        mars_hemispheres_entry[f"hemisphere_name_{index}"] = image_title 
        mars_hemispheres_entry["hemisphere_url"] = image.get_attribute('href')

        browser.back()

    time.sleep(1)

#     mars_db_entry['mars_hemispheres_entry'] = mars_hemispheres_entry

    browser.close()
    
    print(mars_hemispheres_entry)

scrape_mars_hemisphere()

{'hemisphere_name0': 'Cerberus Hemisphere Enhanced', 'hemisphere_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'hemisphere_name1': 'Schiaparelli Hemisphere Enhanced', 'hemisphere_name2': 'Syrtis Major Hemisphere Enhanced', 'hemisphere_name3': 'Valles Marineris Hemisphere Enhanced'}
